# LiDAR Image Overlay
This notebook uses the frontier device extrinsics to generate image-lidar overlays. 

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from sensor import Sensor
from scipy.spatial.transform import Rotation as R
import numpy as np
import helper_functions
import cv2
import open3d as o3d
import os

In [ ]:
if not os.path.exists("sample/2025-08-06_15-47-37_rec000_small/"):
    import subprocess
    import zipfile
    print("Downloading...", end="")
    subprocess.run(
        [
            "wget",
            "https://www.ipb.uni-bonn.de/data/digiforest-dataset/data/calibration_example.zip",
            "-q",
            "-nc",
            "-P",
            "sample"
        ],
        check=True,
    )
    print("finished.")

    print("Extracting files...", end="")
    with zipfile.ZipFile("sample/calibration_example.zip", "r") as zip_file:
        zip_file.extractall("sample/")
    print("finished.")

# Set sensor config file and slam folder used for testing overlays.
sensor_yaml = "extrinsics/sensors_frn015-mar23.yaml"
slam_folder = "sample/2025-08-06_15-47-37_rec000_small"

# Load device extrinsics
sensor = Sensor.load_config(Path(sensor_yaml))

In [ ]:
# load a lidar and camera image pair
pcd_folder = os.path.join(slam_folder, "raw_data/raw_clouds")
image_folder = os.path.join(slam_folder , "raw_data/raw_images/cam_left")

matches = helper_functions.find_closest_images(pcd_folder, image_folder)

# Iterate through the matches
for pcd_file, image_file in matches:
    print(f"Processing Point Cloud: {pcd_file} -> Closest Image: {image_file}")

    lidar = o3d.io.read_point_cloud(slam_folder + "/raw_data/raw_clouds/" + pcd_file)

    for camera in sensor.cameras:
        image = cv2.imread(slam_folder + "/raw_data/raw_images/" + camera.label + "/" + image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # transform lidar points from base frame to camera frame
        lidar_points = np.asarray(lidar.points)
        L_T_LB = sensor.tf.get_transform("base", "lidar") 
        lidar_points_in_lidar = helper_functions.transform_points_tf_matrix(lidar_points.copy(), L_T_LB)

        #  convert lidar points from lidar to camera frame
        C_T_CL = sensor.tf.get_transform("lidar", camera.label)
        lidar_points_in_camera = helper_functions.transform_points_tf_matrix(lidar_points_in_lidar.copy(), C_T_CL)

        # transform lidar points from base frame to camera frame
        lidar_points = np.asarray(lidar.points)
        C_T_CB = sensor.tf.get_transform("base", camera.label)
        lidar_points_in_camera = helper_functions.transform_points_tf_matrix(lidar_points.copy(), C_T_CB)

        # Ensure intrinsics and distortion parameters are numpy arrays with correct shapes (Ate these correct)
        K = np.array(camera.fisheye_intrinsics, dtype=np.float64).reshape(3, 3)
        D = np.array(camera.fisheye_extra_params, dtype=np.float64).reshape(-1, 1)  # OpenCV expects (4,1) or (N,1)

        #(d) Reproject the LiDAR points onto the current image using the fisheye camera projection model: (can be changed to a different reprojection model at a latger stage)
        lidar_points_image, _ = cv2.fisheye.projectPoints(lidar_points_in_camera.reshape(-1, 1, 3).astype(np.float32), np.array([[0.0], [0.0], [0.0]]),np.array([[0.0], [0.0], [0.0]]), K, D)
        lidar_points_image = lidar_points_image.reshape(-1, 2)
        to_keep = np.where((lidar_points_image[:, 0] > 0) & (lidar_points_image[:, 1] > 0) & (lidar_points_image[:, 0] < image.shape[1]) & (lidar_points_image[:, 1] < image.shape[0]) & (lidar_points[:, 2] > 0))
        lidar_points_image = np.round(lidar_points_image[to_keep]).astype(np.uint32) # Convert to (row, column) form (x, y) which is the OpenCV standard.
        depth_pts = lidar_points[to_keep, 2].copy() #Extract the depth associated with each of the image lidar points.

        #(e) Visualize if enabled:
        image_temp = image.copy()

        depth_norm = cv2.normalize(depth_pts, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        depth_colors = cv2.applyColorMap(depth_norm, cv2.COLORMAP_JET).reshape(-1, 3)

        # Draw blobs for each point
        for i in range(lidar_points_image.shape[0]):
            x, y = lidar_points_image[i]
            color = tuple(int(c) for c in depth_colors[i])  # Convert to tuple for cv2
            cv2.circle(image_temp, (x, y), radius=3, color=color, thickness=-1)  # filled circle

        output_dir = os.path.join(slam_folder, "overlay", camera.label)
        os.makedirs(output_dir, exist_ok=True)

        output_path = os.path.join(output_dir, image_file)
        print(f"Saving overlay image for {camera.label} at {output_path}")
        cv2.imwrite(output_path, cv2.cvtColor(image_temp, cv2.COLOR_RGB2BGR))
        # cv2.imshow("lidar", cv2.cvtColor(image_temp, cv2.COLOR_RGB2BGR))
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()